In [1]:
#Library 
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv('/Users/faustin/Downloads/merged_datasets_with_events.csv',sep =',')

# Deleting the covid period
covid_start = '2020-03-01'
covid_stop = '2022-01-01'


df = df[(df['WORK_DATE'] < covid_start) & (df['WORK_DATE'] > covid_stop)]



In [ ]:
#Train test split 
np.random.seed(42)
X_train, X_test 